## **Tech Challenge**

### **Projeto**

Entender como foi o comportamento da população na época da pandemia da COVID-19 e quais indicadores seriam importantes para o planejamento, caso haja um novo surto da doença, utilizando o estudo do PNAD-COVID 19 do IBGE para termos respostas ao problema proposto, pois são dados confiáveis, porém, não será necessário utilizar todas as perguntas realizadas na pesquisa para enxergar todas as oportunidades ali postas, mas há dados triviais que precisam estar no projeto, pois auxiliam muito na análise dos dados:

• Características clínicas dos sintomas;  
• Características da população;  
• Características econômicas da sociedade.  

Dessa forma, acessar os dados do PNAD-COVID-19 do IBGE (https://covid19.ibge.gov.br/pnad-covid/) e organizar esta base para análise, utilizando Banco de Dados em Nuvem e trazendo as seguintes características:

a. Utilização de no máximo 20 questionamentos realizados na pesquisa;  
b. Utilizar 3 meses para construção da solução;  
c. Caracterização dos sintomas clínicos da população;  
d. Comportamento da população na época da COVID-19;  
e. Características econômicas da Sociedade;  

Com objetivo de trazer uma breve análise dessas informações, como foi a organização do banco, as perguntas selecionadas para a resposta do problema e quais seriam as principais ações que o hospital deverá tomar em caso de um novo surto de COVID-19

### Arquitetura de Dados

![Arquitetura](https://raw.githubusercontent.com/RicardViana/fiap-Big-Data/main/arquiteteura.jpeg)

1) Dados utilizado referente a Pesquisa Nacional por Amostra de Domicílios (PNAD COVID19) através do link [PNAD](https://www.ibge.gov.br/estatisticas/investigacoes-experimentais/estatisticas-experimentais/27946-divulgacao-semanal-pnadcovid1?t=microdados&utm_source=covid19&utm_medium=hotsite&utm_campaign=covid_19) *
2) Devido a volumetria dos dados, é feito o armazenamento no AWS S3 e usando camandas medalhão
3) Realizar o carregamento dos da camanda GOLD para o banco de dados PostgreSQL criado via AWS RDS
4) Construir a analise utilizando o Power BI conectado ao Data Mart

\* Os dados foram armazenados no GitHub para a construção desse trabalho

### **Observações**

#### **VS Code**

Para correto funcionamento, sempre reiniciar o Kernel antes de rodar o código

#### **Bibliotecas**

Para correto funcionamento, caso não tenha instalado as seguintes bibliotes: 

- `requests`
- `pandas`  
- `boto3`  
- `python-dotenv`  
- `SQLAlchemy`  
- `psycopg2-binary`  

Realizar a instalação via PIP INSTALL (https://pypi.org/) ou CONDA-FORGE (https://anaconda.org/conda-forge)

#### **Conexões**

Para realizar as conexões é utilizado um arquivo .env salvo no mesmo local do projeto e com a seguinte estrutura

```ini
# Credenciais do PostgreSQL PNAD
POSTGRES_USER_PNAD=
POSTGRES_PASSWORD_PNAD=
POSTGRES_HOST_PNAD=
POSTGRES_PORT_PNAD=5432
POSTGRES_DB_PNAD=

# Credenciais AWS
aws_access_key_id=
aws_secret_access_key=
aws_session_token=

region=us-east-1
output=json
```

#### **Configuração do PostgreSQL na AWS RDS**

##### 1. **Criar instância RDS com PostgreSQL (SandBox)**

1. Acesse o console AWS → [https://us-east-1.console.aws.amazon.com/rds/home?region=us-east-1#](https://console.aws.amazon.com/rds/)
2. Clique em **Criar banco de dados**
3. Selecione:
   - **Tipo de banco:** PostgreSQL
   - **Versão:** PostgreSQL 15 (ou mais recente)
   - **Modelo de uso:** SandBox
   - **Identificador da instância:** `postgres-tc`
   - **Usuário:** `postgres`
   - **Senha:** crie uma senha segura
4. Tipo de instância: `db.t3.micro`
5. Armazenamento: 20 GB (SSD General Purpose)
6. **Acesso público:** Habilitado (Sim)
7. **Nome do banco de dados inicial:** `pnad_covid`
8. Clique em **Criar banco de dados**

##### **2. Liberar o IP na VPC / Grupo de Segurança (Security Group)**

1. Vá para **EC2 > Grupos de Segurança**
2. Encontre o grupo associado à instância RDS
3. Clique em **Editar regras de entrada**
4. Adicione uma nova regra:
   - Tipo: `PostgreSQL`
   - Porta: `5432`
   - Origem: `Seu IP` (ou `0.0.0.0/0` temporariamente para teste – cuidado com isso em produção)
5. Salve as alterações.

✅ Agora o acesso externo ao banco estará liberado para seu IP

**Obs.: Essa configuração será feita apenas uma vez no VPC Default**

### **Códigos**

#### **Importar bibliotecas**

In [1]:
# Importar biblioteca completa
import requests
import zipfile
import io
import pandas as pd
import os
import boto3
import sys
import time 

# Importar algo especifico de uma biblioteca
from dotenv import load_dotenv
from sqlalchemy import create_engine, text
from botocore.exceptions import BotoCoreError, ClientError
from psycopg2 import sql

#### **Funções (DEF)**

In [2]:
# Testar a conexão ao banco de dados
def test_connection(engine):

    try:
        with engine.connect() as connection:
            
            # Testar a versão do PostgreSQL
            result = connection.execute(text("SELECT version();"))
            versao = result.fetchone()
            print("✅ Conectado com sucesso:", versao[0])

            # Listar as tabelas no schema público
            result = connection.execute(text("""
                SELECT table_name
                FROM information_schema.tables
                WHERE table_schema = 'public';
            """))
            tabelas = result.fetchall()
            print("📄 Tabelas no banco:")
            for tabela in tabelas:
                print("-", tabela[0])

    except Exception as e:
        print("❌ Erro ao executar comandos:", e)
        sys.exit()


In [ ]:
# Calcular o tempo de execução
def calcular_tempo_execucao(tempo_inicio, tempo_final):
    """
    Calcula e exibe o tempo total de execução de um processo.

    Parâmetros:
    tempo_inicio (float): O tempo de início capturado com time.time()
    tempo_final (float): O tempo final capturado com time.time()

    A função não retorna nada, apenas imprime o resultado formatado.
    """
    # Calcula a diferença total em segundos
    tempo_total_segundos = tempo_final - tempo_inicio

    # Converte o total de segundos para minutos e segundos
    minutos, segundos = divmod(tempo_total_segundos, 60)

    # Imprime o resultado
    print(f"\nTempo total de execução do código:")
    print(f"{int(minutos)} minutos e {int(segundos)} segundos")

#### **Variaveis**

In [4]:
# Nome do bucket S3 e subpastas
s3_bucket = 'tech-challenge-fase3'
s3_subpasta_bronze = 'bronze'
s3_subpasta_silver = 'silver'
s3_subpasta_gold = 'gold'

silver_prefix = f's3://{s3_bucket}/{s3_subpasta_silver}/'
gold_prefix = f's3://{s3_bucket}/{s3_subpasta_gold}/'

# Nome arquivo .parquet
nome_arquivo_gold = 'pnad_final_tratado.parquet'

# Caminho de saida e entrada camada silver
caminho_saida_silver = silver_prefix + 'pnad_consolidado_enriquecido.parquet'
caminho_entrada_silver = silver_prefix + 'pnad_consolidado_enriquecido.parquet'
caminho_saida_gold = gold_prefix + 'pnad_final_tratado.parquet'
caminho_completo_gold = gold_prefix + nome_arquivo_gold

# Caminho do Github com dados do PNAD
api_url = 'https://api.github.com/repos/RicardViana/fiap-Big-Data/contents/PNAD-COVID/Microdados'

# Caminho do Github com dados do código IBGE UF
link_codigo_uf = 'https://raw.githubusercontent.com/RicardViana/tabela-uf-ibge/refs/heads/main/codigo_uf.csv'

# Nome da tabela no PostgreSQL
nome_tabela = 'questionario_pnad_covid'


#### **Configuração AWS e Banco de Dados**

In [5]:
# Carregar as credencias do .env
load_dotenv()

# Configuração storage_options
storage_options = {
    "key": os.getenv('AWS_ACCESS_KEY_ID'),
    "secret": os.getenv('AWS_SECRET_ACCESS_KEY'),
    "token": os.getenv('AWS_SESSION_TOKEN')
}

# Configuração S3
s3_client = boto3.client(
    's3',
    aws_access_key_id=os.getenv('AWS_ACCESS_KEY_ID'),
    aws_secret_access_key=os.getenv('AWS_SECRET_ACCESS_KEY'),
    aws_session_token=os.getenv('AWS_SESSION_TOKEN'),
    region_name=os.getenv('AWS_REGION')
)

# Credenciais do PostgreSQL
usuario_pg = os.getenv("POSTGRES_USER_PNAD")
senha_pg = os.getenv("POSTGRES_PASSWORD_PNAD")
host_pg = os.getenv("POSTGRES_HOST_PNAD")
porta_pg = os.getenv("POSTGRES_PORT_PNAD")
banco_pg = os.getenv("POSTGRES_DB_PNAD")

#### **Validar Conexões e criar engine banco de dados**

In [6]:
# Validar conexão com a AWS através do .env
load_dotenv()

try:
    sts_client = boto3.client(
        'sts',
        aws_access_key_id=os.getenv('AWS_ACCESS_KEY_ID'),
        aws_secret_access_key=os.getenv('AWS_SECRET_ACCESS_KEY'),
        aws_session_token=os.getenv('AWS_SESSION_TOKEN'),
        region_name=os.getenv('AWS_REGION')
    )
    
    identity = sts_client.get_caller_identity()
    print("✅ Conectado à conta\n")
    print("UserId:", identity["UserId"])
    print("Account:", identity["Account"])
    print("Arn:", identity["Arn"])

except (BotoCoreError, ClientError) as e:
    print("❌ Erro ao conectar à AWS. Verifique suas credenciais e tente novamente.")
    print("Detalhes do erro:", e)

✅ Conectado à conta

UserId: AROAWIUHO6CVTPL5SPXB7:user4308167=ricardviana1@gmail.com
Account: 430854566059
Arn: arn:aws:sts::430854566059:assumed-role/voclabs/user4308167=ricardviana1@gmail.com


In [7]:
# Criar engine com banco 
engine = create_engine(f"postgresql+psycopg2://{usuario_pg}:{senha_pg}@{host_pg}:{porta_pg}/{banco_pg}")

# Testar a conexão
test_connection(engine)

✅ Conectado com sucesso: PostgreSQL 17.4 on x86_64-pc-linux-gnu, compiled by gcc (GCC) 12.4.0, 64-bit
📄 Tabelas no banco:
- questionario_pnad_covid


#### **ETL**

In [8]:
# Registrar o tempo inicio do código
variavel_tempo_inicio = time.time()

In [9]:
# Criar Bucket e subpastas (camadas medalhões)

# Lista com o nome das subpastas
pastas_a_criar = [s3_subpasta_bronze, s3_subpasta_silver, s3_subpasta_gold]

# Armazenar a região da conexão com a AWS
aws_region = s3_client.meta.region_name

print(f"\nIniciando Processo de Validação e Criação no S3")

print(f"\nValidando o bucket '{s3_bucket}'")
bucket_pronto = False

# Validar se o Bucket esta criado
try:
    s3_client.head_bucket(Bucket=s3_bucket)
    print(f"➡️  Bucket '{s3_bucket}' já existe")
    bucket_pronto = True

except ClientError as e:
    
    if e.response['Error']['Code'] == '404':
        print(f"Bucket '{s3_bucket}' não encontrado. Tentando criar")

        try:
            if aws_region == "us-east-1":
                s3_client.create_bucket(Bucket=s3_bucket)

            else:
                location = {'LocationConstraint': aws_region}
                s3_client.create_bucket(
                    Bucket=s3_bucket,
                    CreateBucketConfiguration=location
                )
            print(f"✅ Bucket '{s3_bucket}' criado com sucesso!")
            bucket_pronto = True

        except Exception as create_e:
            print(f"❌ Falha ao tentar criar o bucket: {create_e}")

    else:
        print(f"❌ Erro de permissão ou outro problema ao verificar o bucket: {e}")

# Validar se as subpastas estão criadas
if bucket_pronto:
    print(f"\nValidando as subpastas no bucket '{s3_bucket}'")

    for nome_pasta in pastas_a_criar:
        chave_pasta = nome_pasta if nome_pasta.endswith('/') else nome_pasta + '/'
        
        try:
            s3_client.head_object(Bucket=s3_bucket, Key=chave_pasta)
            print(f"➡️ Pasta '{chave_pasta}' já existe")

        except ClientError as e:

            if e.response['Error']['Code'] == '404':
                try:
                    s3_client.put_object(
                        Bucket=s3_bucket,
                        Key=chave_pasta,
                        Body=''
                    )
                    print(f"✅ Pasta '{chave_pasta}' criada com sucesso")

                except Exception as create_e:
                    print(f"❌ Falha ao TENTAR CRIAR a pasta '{chave_pasta}': {create_e}")

            else:
                print(f"❌ Erro ao verificar a pasta '{chave_pasta}': {e}")
else:
    print("\nCriação das pastas abortada, pois houve um problema com o bucket")

print("\nProcesso Finalizado")


Iniciando Processo de Validação e Criação no S3

Validando o bucket 'tech-challenge-fase3'
➡️  Bucket 'tech-challenge-fase3' já existe

Validando as subpastas no bucket 'tech-challenge-fase3'
➡️ Pasta 'bronze/' já existe
➡️ Pasta 'silver/' já existe
➡️ Pasta 'gold/' já existe

Processo Finalizado


In [10]:
# Carregar os dados para camada Bronze

# Realizar a consulta a URL
print(f"Buscando lista de arquivos em: {api_url}")

response = requests.get(api_url) # Fazer requisição HTTP
response.raise_for_status() # Verificar se a requisição foi bem feita
files = response.json() # Transformar em uma lista de dicionario

# Itera sobre cada item no diretório do GitHub
for file_info in files:

    if file_info['name'].endswith('.zip'):

        zip_name = file_info['name']
        zip_url = file_info['download_url']

        print(f"\nProcessando: {zip_name}")
        
        try:
            r_zip = requests.get(zip_url)
            r_zip.raise_for_status()
            zip_content = io.BytesIO(r_zip.content) # Arquivo temporario salvo em memoria

            with zipfile.ZipFile(zip_content) as z:
                csv_filename = [f for f in z.namelist() if f.endswith('.csv')][0]

                print(f"Arquivo CSV encontrado: {csv_filename}")

                csv_content_bytes = z.read(csv_filename)
                caminho_completo_s3 = f"{s3_subpasta_bronze}/{csv_filename}"

                print(f"Enviando para o S3 em: '{caminho_completo_s3}'")

                # Enviar arquivo .csv para o S3
                s3_client.put_object(
                    Bucket=s3_bucket, # Nome do bucket
                    Key=caminho_completo_s3, # Caminho completo do arquivo
                    Body=csv_content_bytes # Arquivo que vai ser carregado
                )

                print(f"✅ Sucesso! Arquivo enviado para s3://{s3_bucket}/{caminho_completo_s3}")
        
        # Caso ocorra algum erro
        except Exception as e:
            print(f"Erro: Ocorreu um erro inesperado ao processar {zip_name}. Erro: {e}")
            sys.exit()
            
print("\nProcesso para o S3 concluído!")

Buscando lista de arquivos em: https://api.github.com/repos/RicardViana/fiap-Big-Data/contents/PNAD-COVID/Microdados

Processando: PNAD_COVID_052020.zip
Arquivo CSV encontrado: PNAD_COVID_052020.csv
Enviando para o S3 em: 'bronze/PNAD_COVID_052020.csv'
✅ Sucesso! Arquivo enviado para s3://tech-challenge-fase3/bronze/PNAD_COVID_052020.csv

Processando: PNAD_COVID_062020.zip
Arquivo CSV encontrado: PNAD_COVID_062020.csv
Enviando para o S3 em: 'bronze/PNAD_COVID_062020.csv'
✅ Sucesso! Arquivo enviado para s3://tech-challenge-fase3/bronze/PNAD_COVID_062020.csv

Processando: PNAD_COVID_072020.zip
Arquivo CSV encontrado: PNAD_COVID_072020.csv
Enviando para o S3 em: 'bronze/PNAD_COVID_072020.csv'
✅ Sucesso! Arquivo enviado para s3://tech-challenge-fase3/bronze/PNAD_COVID_072020.csv

Processando: PNAD_COVID_082020.zip
Arquivo CSV encontrado: PNAD_COVID_082020.csv
Enviando para o S3 em: 'bronze/PNAD_COVID_082020.csv'
✅ Sucesso! Arquivo enviado para s3://tech-challenge-fase3/bronze/PNAD_COVID_08

In [11]:
# Ler arquivos CSV Codigo IBGE e gerar Data Frame
print(f"\nLendo tabela com os códigos do IBGE de: {link_codigo_uf}")
df_uf = pd.read_csv(link_codigo_uf, sep=",")
print("Data Frame criado com sucesso")


Lendo tabela com os códigos do IBGE de: https://raw.githubusercontent.com/RicardViana/tabela-uf-ibge/refs/heads/main/codigo_uf.csv
Data Frame criado com sucesso


In [12]:
# Carregar dados para camada silver

# Ler arquivos CSV PNAD COVID do S3, consolidar e gerar Data Frame
response = s3_client.list_objects_v2(Bucket=s3_bucket, Prefix=f'{s3_subpasta_bronze}/') # Listar todos os objetos (arquivos) dentro do S3
bronze_files = [obj['Key'] for obj in response.get('Contents', []) if obj['Key'].endswith('.csv')] # Pegar apenas arquivos .csv
lista_de_dataframes = []

print("Lendo arquivos da camada Bronze no S3")

# Iterar sobre os arquivos
for file_key in bronze_files:

    file_path = f"s3://{s3_bucket}/{file_key}"
    print(f"Lendo {file_path}")
    df_temp = pd.read_csv(file_path, storage_options=storage_options, sep=',')
    lista_de_dataframes.append(df_temp)

# Consolidar os data frame
df_consolidado = pd.concat(lista_de_dataframes, ignore_index=True)
print(f"\nConsolidação concluída e {len(df_consolidado)} linhas lidas da camada Bronze")

# Relacionar os Data Frame
print("\nIniciando enriquecimento dos dados com merge com o Data Frame df_uf")
df_silver = pd.merge(
    df_consolidado,
    df_uf,
    how='left',
    left_on='UF', 
    right_on='Código'
)

# Remover colunas não necessarias e ajustar o nome das colunas
df_silver = df_silver.drop(columns=["Código"])
df_silver = df_silver.rename(columns={"UF_x": "UF", "UF_y": "UF_Nome", "Região": "Regiao"})
print("Enriquecimento concluído")

# Ver o resultado final
print("\nData Frame com os primeiros registros")
display(df_silver.head(5))

print("Data Frame com os ultimos registros")
display(df_silver.tail(5))


Lendo arquivos da camada Bronze no S3
Lendo s3://tech-challenge-fase3/bronze/PNAD_COVID_052020.csv
Lendo s3://tech-challenge-fase3/bronze/PNAD_COVID_062020.csv
Lendo s3://tech-challenge-fase3/bronze/PNAD_COVID_072020.csv
Lendo s3://tech-challenge-fase3/bronze/PNAD_COVID_082020.csv
Lendo s3://tech-challenge-fase3/bronze/PNAD_COVID_092020.csv
Lendo s3://tech-challenge-fase3/bronze/PNAD_COVID_102020.csv
Lendo s3://tech-challenge-fase3/bronze/PNAD_COVID_112020.csv

Consolidação concluída e 2650459 linhas lidas da camada Bronze

Iniciando enriquecimento dos dados com merge com o Data Frame df_uf
Enriquecimento concluído

Data Frame com os primeiros registros


,Ano,UF,CAPITAL,RM_RIDE,V1008,V1012,V1013,V1016,Estrato,UPA,...,F002A2,F002A3,F002A4,F002A5,A006A,A006B,A007A,UF_Nome,Sigla,Regiao
0,2020,11,11.0,NaN,1,4,5,1,1110011,110015970,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Rondônia,RO,Norte
1,2020,11,11.0,NaN,1,4,5,1,1110011,110015970,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Rondônia,RO,Norte
2,2020,11,11.0,NaN,1,4,5,1,1110011,110015970,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Rondônia,RO,Norte
3,2020,11,11.0,NaN,1,4,5,1,1110011,110015970,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Rondônia,RO,Norte
4,2020,11,11.0,NaN,3,2,5,1,1110011,110015970,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Rondônia,RO,Norte


Data Frame com os ultimos registros


,Ano,UF,CAPITAL,RM_RIDE,V1008,V1012,V1013,V1016,Estrato,UPA,...,F002A2,F002A3,F002A4,F002A5,A006A,A006B,A007A,UF_Nome,Sigla,Regiao
2650454,2020,53,53.0,NaN,6,3,11,7,5310220,530009738,...,1.0,1.0,2.0,1.0,NaN,NaN,NaN,Distrito Federal,DF,Centro-Oeste
2650455,2020,53,53.0,NaN,6,3,11,7,5310220,530009738,...,1.0,1.0,2.0,1.0,NaN,NaN,NaN,Distrito Federal,DF,Centro-Oeste
2650456,2020,53,53.0,NaN,6,3,11,7,5310220,530009738,...,1.0,1.0,2.0,1.0,1.0,4.0,NaN,Distrito Federal,DF,Centro-Oeste
2650457,2020,53,53.0,NaN,10,2,11,7,5310220,530009738,...,1.0,1.0,1.0,1.0,NaN,NaN,NaN,Distrito Federal,DF,Centro-Oeste
2650458,2020,53,53.0,NaN,10,2,11,7,5310220,530009738,...,1.0,1.0,1.0,1.0,NaN,NaN,NaN,Distrito Federal,DF,Centro-Oeste


In [13]:
# Salvar Data Frame df_silver na camada silver
print(f"\nSalvando dados da camada Silver em: {caminho_saida_silver}")

df_silver.to_parquet(
    caminho_saida_silver,
    index=False,
    storage_options=storage_options
)

print("✅ Sucesso! Camada Silver criada e salva no S3")


Salvando dados da camada Silver em: s3://tech-challenge-fase3/silver/pnad_consolidado_enriquecido.parquet
✅ Sucesso! Camada Silver criada e salva no S3


In [14]:
# Tratar os dados para a camada Gold
print(f"Lendo dados da camada Silver de: {caminho_entrada_silver}")

try:
    df_silver = pd.read_parquet(caminho_entrada_silver, storage_options=storage_options)
    print(f"Leitura concluída onde o dataFrame contem {len(df_silver)} linhas e {len(df_silver.columns)} colunas")

except Exception as e:
    print(f"❌ Erro ao ler o arquivo da camada Silver. Verifique o caminho e as permissões. Detalhes: {e}")
    sys.exit()

# Copiar e padronizar colunas para minúsculas
df_estruturado = df_silver.copy()
df_estruturado.columns = df_estruturado.columns.str.lower()

# Colunas fixas (com descrição)
colunas_fixas = [
    'ano',       # Ano
    'v1013',     # Mês do Ano
    'v1012',     # Semana do Mês
    'uf',        # Sigla da Unidade da Federação
    'capital',   # Capital do Estado
    'rm_ride',   # Região Metropolitana e Região Administrativa Integrada de Desenvolvimento
    'uf_nome',   # Nome do UF
    'sigla',     # Sigla do UF
    'regiao',    # Região
]

# Colunas desejadas (com descrição)
colunas_desejadas = [
    'a002',      # Idade
    'a003',      # Sexo
    'a004',      # Raça ou Cor
    'a006b',     # Você está tendo aulas presenciais?
    'b008',      # O(A) Sr(a) fez algum teste para saber se estava infectado(a) pelo coronavírus?
    'b009a',     # Fez o exame coletado com cotonete na boca e/ou nariz (SWAB)?
    'b009c',     # Fez o exame de coleta de sangue através de furo no dedo?
    'b009e',     # Fez o exame de coleta de sangue através da veia do braço?
    'a005',      # Escolaridade
    'a006',      # Frequenta escola
    'a006a',     # A escola/faculdade que frequenta é pública ou privada?
    'b0011',     # Na semana passada teve febre?
    'b0012',     # Na semana passada teve tosse?
    'b0013',     # Na semana passada teve dor de garganta?
    'b0014',     # Na semana passada teve dificuldade para respirar?
    'b0015',     # Na semana passada teve dor de cabeça?
    'b0016',     # Na semana passada teve dor no peito?
    'b0017',     # Na semana passada teve náusea?
    'b0018',     # Na semana passada teve nariz entupido ou escorrendo?
    'b0019',     # Na semana passada teve fadiga?
    'b00110',    # Na semana passada teve dor nos olhos?
    'b00111',    # Na semana passada teve perda de cheiro ou sabor?
    'b00112',    # Na semana passada teve dor muscular?
    'b00113',    # Na semana passada teve diarreia?
]

# Unir listas mantendo a ordem e sem duplicar
todas_colunas = list(dict.fromkeys([*colunas_fixas, *colunas_desejadas]))

# Manter apenas as colunas que existem no DataFrame após padronização
colunas_existentes = [c for c in todas_colunas if c in df_estruturado.columns]

# Filtro com os 3 últimos valores de v1013
if 'v1013' in df_estruturado.columns:
    # Coletar valores únicos (excluindo NaN), ordenar e pegar os 3 últimos
    ultimos_3 = sorted(pd.unique(df_estruturado['v1013'].dropna()))[-3:]

    # Filtrar registros pertencentes aos 3 últimos meses e selecionar colunas
    estrutura_final = (
        df_estruturado[df_estruturado['v1013'].isin(ultimos_3)][colunas_existentes]
        .reset_index(drop=True)
    )
else:
    # Se não houver v1013, apenas seleciona as colunas existentes
    estrutura_final = df_estruturado[colunas_existentes].reset_index(drop=True)

# Avisos sobre colunas ausentes
faltantes = [c for c in todas_colunas if c not in df_estruturado.columns]
if faltantes:
    print(f"Atenção: estas colunas não foram encontradas e ficaram de fora: {faltantes}")

# Ver o resultado final
print("\nData Frame com os primeiros registros")
display(estrutura_final.head(5))

print("Data Frame com os ultimos registros")
display(estrutura_final.tail(5))


Lendo dados da camada Silver de: s3://tech-challenge-fase3/silver/pnad_consolidado_enriquecido.parquet
Leitura concluída onde o dataFrame contem 2650459 linhas e 151 colunas

Data Frame com os primeiros registros


,ano,v1013,v1012,uf,capital,rm_ride,uf_nome,sigla,regiao,a002,...,b0014,b0015,b0016,b0017,b0018,b0019,b00110,b00111,b00112,b00113
0,2020,9,4,11,11.0,NaN,Rondônia,RO,Norte,36,...,2,2,2,2,2,2,2,2,2,2.0
1,2020,9,4,11,11.0,NaN,Rondônia,RO,Norte,30,...,2,2,2,2,2,2,2,2,2,2.0
2,2020,9,4,11,11.0,NaN,Rondônia,RO,Norte,13,...,2,2,2,2,2,2,2,2,2,2.0
3,2020,9,4,11,11.0,NaN,Rondônia,RO,Norte,11,...,2,2,2,2,2,2,2,2,2,2.0
4,2020,9,1,11,11.0,NaN,Rondônia,RO,Norte,57,...,2,2,2,2,2,2,2,2,2,2.0


Data Frame com os ultimos registros


,ano,v1013,v1012,uf,capital,rm_ride,uf_nome,sigla,regiao,a002,...,b0014,b0015,b0016,b0017,b0018,b0019,b00110,b00111,b00112,b00113
1149192,2020,11,3,53,53.0,NaN,Distrito Federal,DF,Centro-Oeste,45,...,2,2,2,2,2,2,2,2,2,2.0
1149193,2020,11,3,53,53.0,NaN,Distrito Federal,DF,Centro-Oeste,22,...,2,2,2,2,2,2,2,2,2,2.0
1149194,2020,11,3,53,53.0,NaN,Distrito Federal,DF,Centro-Oeste,16,...,2,2,2,2,2,2,2,2,2,2.0
1149195,2020,11,2,53,53.0,NaN,Distrito Federal,DF,Centro-Oeste,83,...,2,2,2,2,2,2,2,2,2,2.0
1149196,2020,11,2,53,53.0,NaN,Distrito Federal,DF,Centro-Oeste,75,...,2,2,2,2,2,2,2,2,2,2.0


In [15]:
# Carregar os dados para a Gold
print(f"\nSalvando dados da camada Gold em: {caminho_saida_gold}")

# Salvar o data frame como .parquet 
estrutura_final.to_parquet(
    caminho_saida_gold,
    index=False,
    storage_options=storage_options # Parametro necessario para conseguir conectar o Pandas ao AWS
)

print(f"✅ Sucesso! Camada Gold criada com {len(estrutura_final)} linhas e salva no S3.")


Salvando dados da camada Gold em: s3://tech-challenge-fase3/gold/pnad_final_tratado.parquet
✅ Sucesso! Camada Gold criada com 1149197 linhas e salva no S3.


In [16]:
# Criar a tabela no banco de dados PostgreSQL
print("Iniciando a criação do esquema da tabela")

try:
    df_schema = estrutura_final.head(0)
    df_schema.to_sql(
        nome_tabela, 
        con=engine, 
        if_exists='replace', # Dropar a tabela
        index=False)
    
    print(f"✅ Esquema da tabela '{nome_tabela}' criado com sucesso no PostgreSQL!")

    # Adicionar colunas padrão de carga com valores DEFAULT
    print("\nAdicionando colunas de metadados (data_hora_carga, usuario_carga)")

    with engine.connect() as connection:
        comando_sql_data = text(f"""
            ALTER TABLE {nome_tabela}
            ADD COLUMN data_hora_carga TIMESTAMPTZ DEFAULT NOW();
        """)
        
        comando_sql_usuario = text(f"""
            ALTER TABLE {nome_tabela}
            ADD COLUMN usuario_carga VARCHAR(255) DEFAULT CURRENT_USER;
        """)

        connection.execute(comando_sql_data)
        connection.execute(comando_sql_usuario)
        connection.commit()

        print(f"✅ Colunas de metadados adicionadas com sucesso à tabela '{nome_tabela}'.")

except Exception as e:
    print(f"❌ Erro durante a criação ou alteração da tabela: {e}")
    sys.exit()

Iniciando a criação do esquema da tabela
✅ Esquema da tabela 'questionario_pnad_covid' criado com sucesso no PostgreSQL!

Adicionando colunas de metadados (data_hora_carga, usuario_carga)
✅ Colunas de metadados adicionadas com sucesso à tabela 'questionario_pnad_covid'.


In [17]:
# Carregar dados da Gold para PostgreSQL
with engine.connect() as connection:
    try:
        print(f"Lendo dados de: {caminho_completo_gold}")
        df_gold = pd.read_parquet(caminho_completo_gold, storage_options=storage_options)
        print(f"Leitura concluída. DataFrame com {len(df_gold)} linhas.")

        # Obtemos a conexão do psycopg2 a partir da conexão da engine
        raw_conn = connection.connection
        cursor = raw_conn.cursor()

        # Truncar a tabela
        print(f"Limpando a tabela de destino '{nome_tabela}'")
        truncate_query = sql.SQL("TRUNCATE TABLE {}").format(sql.Identifier(nome_tabela))
        cursor.execute(truncate_query)
        
        # Carregar os dados via COPY
        buffer = io.StringIO()
        df_gold.to_csv(buffer, index=False, header=False, sep='\t')
        buffer.seek(0)
        print(f"Iniciando a carga de {len(df_gold)} linhas via COPY")
        colunas_df = list(df_gold.columns)
        cursor.copy_from(buffer, nome_tabela, sep='\t', null='', columns=colunas_df)
        print("Carga via COPY concluída")
        raw_conn.commit()
        cursor.close()
        print(f"✅ Sucesso! Transação efetivada na tabela '{nome_tabela}'")

        # Salvar os dados da tabela do PostgreSQL em um .csv
        print("\nIniciando a exportação dos dados da tabela para .csv")
        sql_query = f"SELECT * FROM {nome_tabela};"
        df_exportar = pd.read_sql_query(sql_query, connection)
        diretorio_script = os.getcwd() 
        nome_arquivo_csv = f"{nome_tabela}_exportado.csv"
        caminho_csv_exportado = os.path.join(diretorio_script, nome_arquivo_csv)
        df_exportar.to_csv(caminho_csv_exportado, index=False, sep=';', encoding='utf-8-sig')
        print("✅ Dados exportados com sucesso!")
        print(f"Localização: {caminho_csv_exportado}")
        
        print("\nProcesso concluído com sucesso!")

    except Exception as e:
        print(f"❌ Ocorreu um erro durante o processo: {e}")
        if 'raw_conn' in locals() and not raw_conn.closed:
             raw_conn.rollback()
             print("Transação revertida (rollback).")

Lendo dados de: s3://tech-challenge-fase3/gold/pnad_final_tratado.parquet
Leitura concluída. DataFrame com 1149197 linhas.
Limpando a tabela de destino 'questionario_pnad_covid'
Iniciando a carga de 1149197 linhas via COPY
Carga via COPY concluída
✅ Sucesso! Transação efetivada na tabela 'questionario_pnad_covid'

Iniciando a exportação dos dados da tabela para .csv
✅ Dados exportados com sucesso!
Localização: c:\Users\ricar\OneDrive\Cursos\Pós Graduação\Data Analytics - FIAP\05 - Fase 3 - Big Data\questionario_pnad_covid_exportado.csv

Processo concluído com sucesso!


In [18]:
# Registrar o tempo final do código
variavel_tempo_final = time.time()

# Gerar o resultado
calcular_tempo_execucao(variavel_tempo_inicio, variavel_tempo_final)

✅ 
Tempo total de execução do código:
8 minutos e 6 segundos
